In [ ]:
# V4: fillna(0)
# deleted the meteo columns with more than 1000 NaN values 

In [204]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
train = pd.read_parquet('/Users/leatinelli/Desktop/X-HEC/msdb-2024/train.parquet')

In [206]:
vacances_paris_2020 = [
    ("2020-10-17", "2020-11-01"),  
    ("2020-12-19", "2021-01-03"),  
    ("2021-02-13", "2021-02-28"),  
    ("2021-04-17", "2021-05-02"), 
    ("2021-07-06", "2021-08-31"), 
]

vacances_paris_2020 = [(pd.to_datetime(debut), pd.to_datetime(fin)) for debut, fin in vacances_paris_2020]

In [208]:
def est_vacances(date, vacances):
    for debut, fin in vacances:
        if debut <= date <= fin:
            return 1  
    return 0  

train["vacances_scolaires"] = train["date"].apply(lambda x: est_vacances(x, vacances_paris_2020))

In [210]:
jours_feries = [
    "2020-11-01",  
    "2020-11-11", 
    "2020-12-25",  
    "2021-01-01",  
    "2021-04-05",  
    "2021-05-01",  
    "2021-05-08",  
    "2021-05-13", 
    "2021-05-24",  
    "2021-07-14",  
    "2021-08-15", 
]
jours_feries = pd.to_datetime(jours_feries)

In [212]:
train["jour_ferie"] = train["date"].apply(lambda x: 1 if x in jours_feries else 0)

In [213]:
confinements = [
    ("2020-10-30", "2020-12-15", "confinement"),
    ("2021-03-20", "2021-04-17", "confinement"), 
]

couvre_feux = [
    ("2020-10-17", "2020-12-14", "21:00", "06:00", "couvre-feu"),
    ("2020-12-15", "2021-01-15", "20:00", "06:00", "couvre-feu"),
    ("2021-01-16", "2021-03-19", "18:00", "06:00", "couvre-feu"),
    ("2021-03-20", "2021-06-08", "19:00", "06:00", "couvre-feu"),
    ("2021-06-09", "2021-06-19", "23:00", "06:00", "couvre-feu"),
]

confinements = [(pd.to_datetime(start), pd.to_datetime(end), label) for start, end, label in confinements]
couvre_feux = [(pd.to_datetime(start), pd.to_datetime(end), start_hour, end_hour, label) for start, end, start_hour, end_hour, label in couvre_feux]

In [216]:
train["confinement"] = 0
train["couvre_feu"] = 0

for start, end, label in confinements:
    train.loc[(train["date"] >= start) & (train["date"] <= end), "confinement"] = 1

for start, end, start_hour, end_hour, label in couvre_feux:

    in_couvre_feu_period = (train["date"] >= start) & (train["date"] <= end)

    in_couvre_feu_hours = (train["date"].dt.time >= pd.to_datetime(start_hour).time()) | (train["date"].dt.time <= pd.to_datetime(end_hour).time())

    train.loc[in_couvre_feu_period & in_couvre_feu_hours, "couvre_feu"] = 1
    train.loc[in_couvre_feu_period & in_couvre_feu_hours, "couvre_feu_horaires"] = f"{start_hour}-{end_hour}"

In [218]:
meteo = pd.read_csv('/Users/leatinelli/Desktop/X-HEC/msdb-2024/external_data.csv')

In [220]:
meteo['date'] = pd.to_datetime(meteo['date'])

In [224]:
columns_to_drop = ['cm', 'ch', 'niv_bar', 'geop', 'tn12', 'tn24', 
                   'tx12', 'tx24', 'tminsol', 'sw', 'tw']

# Drop these columns from the dataset
meteo = meteo.drop(columns=columns_to_drop)


In [226]:
df = train.merge(meteo, on='date', how='left')

In [228]:
df = df.drop(columns = ["counter_technical_id", "counter_name", "site_name", "coordinates", "bike_count"])

In [85]:
df

,counter_id,site_id,date,counter_installation_date,latitude,longitude,log_bike_count,vacances_scolaires,jour_ferie,confinement,...,perssfrai,rr1,rr3,rr6,rr12,rr24,phenspe3,nnuage1,ctype1,hnuage1
0,100007049-102007049,100007049,2020-09-01 02:00:00,2013-01-18,48.846028,2.375429,0.000000,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100007049-102007049,100007049,2020-09-01 03:00:00,2013-01-18,48.846028,2.375429,0.693147,0,0,0,...,-30.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,100007049-102007049,100007049,2020-09-01 04:00:00,2013-01-18,48.846028,2.375429,0.000000,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100007049-102007049,100007049,2020-09-01 15:00:00,2013-01-18,48.846028,2.375429,1.609438,0,0,0,...,-30.0,0.0,0.0,0.0,0.0,0.0,NaN,2.0,8.0,1700.0
4,100007049-102007049,100007049,2020-09-01 18:00:00,2013-01-18,48.846028,2.375429,2.302585,0,0,0,...,-60.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,8.0,1700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496876,300014702-353245971,300014702,2021-09-09 06:00:00,2020-11-29,48.839770,2.301980,6.100319,0,0,0,...,-60.0,0.0,0.0,-0.1,1.2,1.2,NaN,7.0,7.0,120.0
496877,300014702-353245971,300014702,2021-09-09 10:00:00,2020-11-29,48.839770,2.301980,4.983607,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496878,300014702-353245971,300014702,2021-09-09 15:00:00,2020-11-29,48.839770,2.301980,5.389072,0,0,0,...,-30.0,0.0,0.0,0.0,0.0,1.2,NaN,3.0,8.0,1380.0
496879,300014702-353245971,300014702,2021-09-09 22:00:00,2020-11-29,48.839770,2.301980,3.091042,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [232]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek  
df['hour'] = df['date'].dt.hour  

df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

df = df.drop(columns=['date'])

df.head()

,counter_id,site_id,counter_installation_date,latitude,longitude,log_bike_count,vacances_scolaires,jour_ferie,confinement,couvre_feu,...,hnuage3,nnuage4,ctype4,hnuage4,year,month,day,day_of_week,hour,is_weekend
0,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,0.000000,0,0,0,0,...,NaN,NaN,NaN,NaN,2020,9,1,1,2,0
1,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,0.693147,0,0,0,0,...,NaN,NaN,NaN,NaN,2020,9,1,1,3,0
2,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,0.000000,0,0,0,0,...,NaN,NaN,NaN,NaN,2020,9,1,1,4,0
3,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,1.609438,0,0,0,0,...,NaN,NaN,NaN,NaN,2020,9,1,1,15,0
4,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,2.302585,0,0,0,0,...,NaN,NaN,NaN,NaN,2020,9,1,1,18,0


In [234]:
df['counter_id_encoded'] = label_encoder.fit_transform(df['counter_id'])
df = df.drop(columns = "counter_id")

In [236]:
df

,site_id,counter_installation_date,latitude,longitude,log_bike_count,vacances_scolaires,jour_ferie,confinement,couvre_feu,couvre_feu_horaires,...,nnuage4,ctype4,hnuage4,year,month,day,day_of_week,hour,is_weekend,counter_id_encoded
0,100007049,2013-01-18,48.846028,2.375429,0.000000,0,0,0,0,NaN,...,NaN,NaN,NaN,2020,9,1,1,2,0,1
1,100007049,2013-01-18,48.846028,2.375429,0.693147,0,0,0,0,NaN,...,NaN,NaN,NaN,2020,9,1,1,3,0,1
2,100007049,2013-01-18,48.846028,2.375429,0.000000,0,0,0,0,NaN,...,NaN,NaN,NaN,2020,9,1,1,4,0,1
3,100007049,2013-01-18,48.846028,2.375429,1.609438,0,0,0,0,NaN,...,NaN,NaN,NaN,2020,9,1,1,15,0,1
4,100007049,2013-01-18,48.846028,2.375429,2.302585,0,0,0,0,NaN,...,NaN,NaN,NaN,2020,9,1,1,18,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496876,300014702,2020-11-29,48.839770,2.301980,6.100319,0,0,0,0,NaN,...,NaN,NaN,NaN,2021,9,9,3,6,0,54
496877,300014702,2020-11-29,48.839770,2.301980,4.983607,0,0,0,0,NaN,...,NaN,NaN,NaN,2021,9,9,3,10,0,54
496878,300014702,2020-11-29,48.839770,2.301980,5.389072,0,0,0,0,NaN,...,NaN,NaN,NaN,2021,9,9,3,15,0,54
496879,300014702,2020-11-29,48.839770,2.301980,3.091042,0,0,0,0,NaN,...,NaN,NaN,NaN,2021,9,9,3,22,0,54


In [238]:
test =  pd.read_parquet('/Users/leatinelli/Desktop/X-HEC/msdb-2024/final_test.parquet')

In [240]:
test["vacances_scolaires"] = test["date"].apply(lambda x: est_vacances(x, vacances_paris_2020))

In [242]:
test["jour_ferie"] = test["date"].apply(lambda x: 1 if x in jours_feries else 0)

In [244]:
test["confinement"] = 0
test["couvre_feu"] = 0

for start, end, label in confinements:
    test.loc[(test["date"] >= start) & (test["date"] <= end), "confinement"] = 1

for start, end, start_hour, end_hour, label in couvre_feux:

    in_couvre_feu_period = (test["date"] >= start) & (test["date"] <= end)

    in_couvre_feu_hours = (test["date"].dt.time >= pd.to_datetime(start_hour).time()) | (test["date"].dt.time <= pd.to_datetime(end_hour).time())

    test.loc[in_couvre_feu_period & in_couvre_feu_hours, "couvre_feu"] = 1
    test.loc[in_couvre_feu_period & in_couvre_feu_hours, "couvre_feu_horaires"] = f"{start_hour}-{end_hour}"

In [246]:
df1 = test.merge(meteo, on='date', how='left')

In [248]:
df1 = df1.drop(columns = ["counter_technical_id", "counter_name", "site_name", "coordinates"])

In [250]:
df1['year'] = df1['date'].dt.year
df1['month'] = df1['date'].dt.month
df1['day'] = df1['date'].dt.day
df1['day_of_week'] = df1['date'].dt.dayofweek  
df1['hour'] = df1['date'].dt.hour  

df1['is_weekend'] = df1['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

df1 = df1.drop(columns=['date'])

df1.head()

,counter_id,site_id,counter_installation_date,latitude,longitude,vacances_scolaires,jour_ferie,confinement,couvre_feu,couvre_feu_horaires,...,hnuage3,nnuage4,ctype4,hnuage4,year,month,day,day_of_week,hour,is_weekend
0,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,2021,9,10,4,1,0
1,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,2021,9,10,4,13,0
2,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,2021,9,10,4,17,0
3,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,2021,9,10,4,19,0
4,100007049-102007049,100007049,2013-01-18,48.846028,2.375429,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,2021,9,10,4,22,0


In [252]:
df1['counter_id_encoded'] = label_encoder.fit_transform(df1['counter_id'])
df1 = df1.drop(columns = "counter_id")

In [254]:
from sklearn.impute import KNNImputer

# Initialiser l'imputateur KNN
imputer = KNNImputer(n_neighbors=2)

In [266]:
df1 = df1.drop(columns = ["counter_installation_date"])

In [269]:
df = df.fillna(0)
df1 = df1.fillna(0)

In [271]:
X = df.drop(columns = "log_bike_count")
y = df["log_bike_count"]

In [273]:
df1 = df1[X.columns]

# Vérifiez que l'ordre des colonnes est identique
print(X.columns)
print(df1.columns)

Index(['site_id', 'latitude', 'longitude', 'vacances_scolaires', 'jour_ferie',
       'confinement', 'couvre_feu', 'numer_sta', 'pmer', 'tend', 'cod_tend',
       'dd', 'ff', 't', 'td', 'u', 'vv', 'ww', 'w1', 'w2', 'n', 'nbas', 'hbas',
       'cl', 'pres', 'tend24', 'raf10', 'rafper', 'per', 'etat_sol',
       'ht_neige', 'ssfrai', 'perssfrai', 'rr1', 'rr3', 'rr6', 'rr12', 'rr24',
       'phenspe1', 'phenspe2', 'phenspe3', 'phenspe4', 'nnuage1', 'ctype1',
       'hnuage1', 'nnuage2', 'ctype2', 'hnuage2', 'nnuage3', 'ctype3',
       'hnuage3', 'nnuage4', 'ctype4', 'hnuage4', 'year', 'month', 'day',
       'day_of_week', 'hour', 'is_weekend', 'counter_id_encoded'],
      dtype='object')
Index(['site_id', 'latitude', 'longitude', 'vacances_scolaires', 'jour_ferie',
       'confinement', 'couvre_feu', 'numer_sta', 'pmer', 'tend', 'cod_tend',
       'dd', 'ff', 't', 'td', 'u', 'vv', 'ww', 'w1', 'w2', 'n', 'nbas', 'hbas',
       'cl', 'pres', 'tend24', 'raf10', 'rafper', 'per', 'etat_sol',
 

In [275]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [277]:
xgb_model = XGBRegressor(
    n_estimators=100,       # Nombre d'arbres (augmenter pour plus de performance)
    learning_rate=0.1,      # Taux d'apprentissage
    max_depth=6,            # Profondeur des arbres
    random_state=42,        # Réplicabilité
    objective='reg:squarederror'  # Régression
)

# Entraîner le modèle
xgb_model.fit(X, y)

# Faire des prédictions
y_pred = xgb_model.predict(df1)

In [279]:
pred4 = pd.DataFrame(y_pred)
pred4.columns = ["log_bike_count"]
pred4.index.name = "Id"
pred4

,log_bike_count
Id,
0,0.431717
1,1.630391
2,2.254465
3,0.961008
4,1.115715
...,...
51435,4.800419
51436,5.119158
51437,5.362166


In [281]:
pred4.to_csv("bike_count_pred_lea_augustin_v'.csv")

In [ ]:
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

ensemble = VotingRegressor([
    ('xgb', XGBRegressor(n_estimators=300, learning_rate=0.05, random_state=42)),
    ('lgbm', LGBMRegressor(n_estimators=300, learning_rate=0.05, random_state=42)),
    ('rf', RandomForestRegressor(n_estimators=300, random_state=42))
])

ensemble.fit(X, y)
y_pred = ensemble.predict(df1)

In [285]:
xgb_model = XGBRegressor(
    n_estimators=150,       # Nombre d'arbres (augmenter pour plus de performance)
    learning_rate=0.1,      # Taux d'apprentissage
    max_depth=6,            # Profondeur des arbres
    random_state=42,        # Réplicabilité
    objective='reg:squarederror'  # Régression
)

# Entraîner le modèle
xgb_model.fit(X, y)

# Faire des prédictions
y_pred = xgb_model.predict(df1)

In [287]:
pred5 = pd.DataFrame(y_pred)
pred5.columns = ["log_bike_count"]
pred5.index.name = "Id"
pred5

,log_bike_count
Id,
0,0.384204
1,1.574756
2,2.214206
3,0.879959
4,1.021113
...,...
51435,4.750231
51436,5.094263
51437,5.428408


In [289]:
pred5.to_csv("bike_count_pred_lea_augustin_v5.csv") # Meilleur score so far 

In [291]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor


In [309]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'min_child_weight': [1, 3, 5]
}

In [297]:
# Initialiser le modèle
xgb_model = XGBRegressor(random_state=42, objective='reg:squarederror')


from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_iter=50,  # Nombre de combinaisons à tester
    random_state=42,
    n_jobs=-1
)

random_search.fit(X, y)
print("Best parameters found: ", random_search.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best parameters found:  {'subsample': 0.8, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 1, 'colsample_bytree': 1.0}


In [299]:
best_model = random_search.best_estimator_

y_pred = best_model.predict(df1)

In [301]:
pred6 = pd.DataFrame(y_pred)
pred6.columns = ["log_bike_count"]
pred6.index.name = "Id"
pred6

,log_bike_count
Id,
0,0.585645
1,1.761980
2,2.376815
3,1.157813
4,1.358615
...,...
51435,4.487203
51436,4.829681
51437,5.158279


In [303]:
pred6.to_csv("bike_count_pred_lea_augustin_v6.csv")

In [312]:
xgb_model = XGBRegressor(
    n_estimators=200,       # Nombre d'arbres (augmenter pour plus de performance)
    learning_rate=0.1,      # Taux d'apprentissage
    max_depth=6,            # Profondeur des arbres
    random_state=42,        # Réplicabilité
    objective='reg:squarederror'  # Régression
)

# Entraîner le modèle
xgb_model.fit(X, y)

# Faire des prédictions
y_pred = xgb_model.predict(df1)

In [314]:
pred7 = pd.DataFrame(y_pred)
pred7.columns = ["log_bike_count"]
pred7.index.name = "Id"
pred7

,log_bike_count
Id,
0,0.346055
1,1.567607
2,2.182717
3,0.836362
4,0.983226
...,...
51435,4.745920
51436,5.056505
51437,5.466711


In [316]:
pred7.to_csv("bike_count_pred_lea_augustin_v7.csv")

In [320]:
xgb_model = XGBRegressor(
    n_estimators=300,       # Nombre d'arbres (augmenter pour plus de performance)
    learning_rate=0.1,      # Taux d'apprentissage
    max_depth=6,            # Profondeur des arbres
    random_state=42,        # Réplicabilité
    objective='reg:squarederror'  # Régression
)

# Entraîner le modèle
xgb_model.fit(X, y)

# Faire des prédictions
y_pred = xgb_model.predict(df1)

In [322]:
pred8 = pd.DataFrame(y_pred)
pred8.columns = ["log_bike_count"]
pred8.index.name = "Id"
pred8

,log_bike_count
Id,
0,0.284348
1,1.561306
2,2.154132
3,0.802513
4,0.943605
...,...
51435,4.729444
51436,5.041428
51437,5.459440


In [324]:
pred8.to_csv("bike_count_pred_lea_augustin_v8.csv")

In [326]:
xgb_model = XGBRegressor(
    n_estimators=350,       # Nombre d'arbres (augmenter pour plus de performance)
    learning_rate=0.1,      # Taux d'apprentissage
    max_depth=6,            # Profondeur des arbres
    random_state=42,        # Réplicabilité
    objective='reg:squarederror'  # Régression
)

# Entraîner le modèle
xgb_model.fit(X, y)

# Faire des prédictions
y_pred = xgb_model.predict(df1)

In [328]:
pred9 = pd.DataFrame(y_pred)
pred9.columns = ["log_bike_count"]
pred9.index.name = "Id"
pred9

,log_bike_count
Id,
0,0.290440
1,1.564242
2,2.160243
3,0.795852
4,0.910598
...,...
51435,4.712652
51436,5.011165
51437,5.456690


In [332]:
pred9.to_csv("bike_count_pred_lea_augustin_v9.csv")